In [1]:
from keras.models import load_model
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense
from keras.applications import ResNet50

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

train_datagen = ImageDataGenerator(
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

In [3]:
train_data_dir = '/home/phu/Documents/terrian_data/'

In [4]:
train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(224, 224),
    batch_size=32, class_mode='categorical')

Found 3827 images belonging to 5 classes.


In [5]:
next(train_generator)

(array([[[[ 60.799297,  64.76331 ,  70.24532 ],
          [ 77.13403 ,  81.99146 ,  87.92017 ],
          [ 92.73272 ,  97.73272 , 103.73272 ],
          ...,
          [ 58.563957,  59.41165 ,  60.824474],
          [ 48.075523,  48.075523,  48.075523],
          [ 55.37654 ,  55.37654 ,  55.37654 ]],
 
         [[ 88.135185,  93.90486 , 100.67453 ],
          [116.36581 , 122.14065 , 128.91548 ],
          [134.84496 , 140.62497 , 147.40497 ],
          ...,
          [ 44.50348 ,  45.351173,  46.763992],
          [ 36.753956,  36.753956,  36.753956],
          [ 43.15486 ,  43.15486 ,  43.15486 ]],
 
         [[147.12213 , 154.12213 , 162.12213 ],
          [153.66191 , 160.66191 , 168.66191 ],
          [151.50513 , 158.05952 , 165.61394 ],
          ...,
          [ 24.89195 ,  25.739645,  27.658903],
          [ 25.355879,  25.824371,  26.761354],
          [ 32.940132,  33.5484  ,  34.76494 ]],
 
         ...,
 
         [[176.82977 , 175.09846 , 162.43175 ],
          [177.138

In [6]:
train_generator.n // 32

119

In [7]:
train_generator.num_classes

5

In [22]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

/home/phu/Documents/anaconda3/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [23]:
from keras import optimizers
optimizer = optimizers.RMSprop(lr=0.00001, rho=0.9, epsilon=None, decay=0.0)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit_generator(train_generator, train_generator.n // 32, epochs=3)

Epoch 1/3
 82/119 [===================>..........] - ETA: 4:25 - loss: 0.7444 - acc: 0.8323

KeyboardInterrupt: 

In [24]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk
